In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from tensorflow.keras.datasets import  cifar10

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse",	"ship",	"truck"]

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess
from tensorflow.keras import layers, models, optimizers

y_train_cat_effnet = to_categorical(y_train, 10)
y_test_cat_effnet = to_categorical(y_test, 10)

# Preprocessing function for EfficientNet
def preprocess(image, label):
    image = tf.image.resize(image, (380, 380))    # EfficientNetB4 input size
    image = effnet_preprocess(image)              # EfficientNet scaling
    return image, label

# Recreate datasets with EfficientNet preprocessing
train_ds_eff = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat_effnet))
test_ds_eff  = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat_effnet))

train_ds_eff = train_ds_eff.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)
test_ds_eff  = test_ds_eff.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)

# Base EfficientNetB4
base_model = EfficientNetB4(input_shape=(380, 380, 3), include_top=False, weights='imagenet')
base_model.trainable = False   # freeze first

# Build model
model_effb4 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile
model_effb4.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train
history_effb4 = model_effb4.fit(train_ds_eff, validation_data=test_ds_eff, epochs=10, callbacks= ModelCheckpoint(filepath='effnet_a_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.7870 - loss: 0.7352
Epoch 1: val_loss improved from inf to 0.23773, saving model to effnet_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 574s 171ms/step - accuracy: 0.7870 - loss: 0.7351 - val_accuracy: 0.9220 - val_loss: 0.2377
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9107 - loss: 0.2695
Epoch 2: val_loss improved from 0.23773 to 0.21996, saving model to effnet_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 474s 150ms/step - accuracy: 0.9107 - loss: 0.2695 - val_accuracy: 0.9279 - val_loss: 0.2200
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9150 - loss: 0.2520
Epoch 3: val_loss improved from 0.21996 to 0.20741, saving model to effnet_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 495s 147ms/step - accuracy: 0.9150 - loss: 0.2520 - val_accuracy: 0.9307 - val_loss: 0.2074
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.9202 - loss: 0.2353
Epoch 4: val_loss

In [10]:
from tensorflow.keras.applications import EfficientNetB5

def preprocess(image, label):
    image = tf.image.resize(image, (456, 456))    # EfficientNetB5 input size
    image = effnet_preprocess(image)
    return image, label

train_ds_eff = tf.data.Dataset.from_tensor_slices((X_train, y_train_cat_effnet))
test_ds_eff  = tf.data.Dataset.from_tensor_slices((X_test, y_test_cat_effnet))

train_ds_eff = train_ds_eff.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)   # smaller batch
test_ds_eff  = test_ds_eff.map(preprocess).batch(16).prefetch(tf.data.AUTOTUNE)

base_model = EfficientNetB5(input_shape=(456, 456, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model_effb5 = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_effb5.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_effb5 = model_effb5.fit(train_ds_eff, validation_data=test_ds_eff, epochs=10, callbacks= ModelCheckpoint(filepath='effnetb5_a_bm.keras',monitor='val_loss',
    save_best_only=True,mode='min',verbose=1))

Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.7936 - loss: 0.7277
Epoch 1: val_loss improved from inf to 0.24621, saving model to effnetb5_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 1054s 316ms/step - accuracy: 0.7936 - loss: 0.7276 - val_accuracy: 0.9168 - val_loss: 0.2462
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.9062 - loss: 0.2752
Epoch 2: val_loss improved from 0.24621 to 0.22419, saving model to effnetb5_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 991s 313ms/step - accuracy: 0.9062 - loss: 0.2752 - val_accuracy: 0.9225 - val_loss: 0.2242
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.9125 - loss: 0.2520
Epoch 3: val_loss improved from 0.22419 to 0.21679, saving model to effnetb5_a_bm.keras
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 981s 313ms/step - accuracy: 0.9125 - loss: 0.2520 - val_accuracy: 0.9250 - val_loss: 0.2168
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.9153 - loss: 0.2476
Epoch 4: v